# Ukládání dat

Úkládání dat

- Homogenní datové struktury
- Heterogenní datové struktury
- Operace nad datovými strukturami
  - Filter
  - Map
  - Reduce
- SQL (SQLAlchemy Python)
- GraphQL (Graphene Python)



## Homogenní datové struktury

Homogenní datové struktury jsou charakterizované "stejností". Velmi často jsou spojovány s tabulkami v SQL databázích. Tyto tabulky mají definici, které je společná pro všechny záznamy.

V jazyku Python je ekvivalentem seznam (list), jehož položky jsou slovníky (dictionary), případně instance specifické třídy (class). V případě dictionary se předpokládá, že všechny mají stejný seznam klíčů, což je důležité pro zpracování dat pomocí funkcí, které mohou předpokládat přítomnost hodnot.

Velmi často se předpokládá, že hodnoty klíčů jsou elementární datové struktury (viz SQL databáze), ale může bát i jinak (NonSQL).

### Příklad testu homogenity dat

In [1]:
import json

with open('rozvrh/data.json') as inputFile:
    data = json.load(inputFile)

In [20]:
events = data['events']
keys = {}
for item in events:
    for key in item.keys():
        keys[key] = True

for item in events:
    for key in keys.keys():
        if not key in item:
            keys[key] = False

print(keys)
print('-'*30)
for key, value in keys.items():
    if value:
        print(key, end=' ')
print()
print('-'*30)
for key, value in keys.items():
    if not value:
        print(key, end=' ')
print()


{'id': True, 'typeId': True, 'startTime': True, 'endTime': True, 'dateCode': True, 'date': True, 'categoryId': True, 'subjectId': False, 'subjectName': False, 'departmentId': False, 'departmentName': False, 'topic': False, 'topicId': False, 'masterId': False, 'timeslotsId': True, 'timeslotsName': True, 'lessonOrder': False, 'lessonFormId': False, 'lessonFormName': False, 'lessonUnit': True, 'lessonsCount': False, 'groupsIds': True, 'groupsNames': True, 'groupsEntryYearsIds': True, 'classroomsIds': True, 'classroomsNames': True, 'classroomsAreasIds': True, 'teachersIds': True, 'teachersNames': True, 'teachersDepartmentsIds': True, 'isLocked': False, 'subtopic': False, 'note': False, 'comment': False, 'supergroupId': False}
------------------------------
id typeId startTime endTime dateCode date categoryId timeslotsId timeslotsName lessonUnit groupsIds groupsNames groupsEntryYearsIds classroomsIds classroomsNames classroomsAreasIds teachersIds teachersNames teachersDepartmentsIds 
------

In [21]:
counter = 1
for item in events:
    for key in keys.keys():
        if not key in item:
            counter = counter + 1
            if counter > 3:
                break
            print(item, '?', key)
            

{'id': '2D2B1C90-F4FF-11EB-9B74-520D00000000', 'typeId': 'F00CB650-9768-11EB-87D2-030800000000', 'startTime': {'hours': 14, 'minutes': 30}, 'endTime': {'hours': 16, 'minutes': 0}, 'dateCode': '2021-11-15', 'date': {'day': 15, 'month': 11, 'year': 2021}, 'categoryId': 1, 'subjectId': 32269, 'subjectName': 'Letecké elektronické systémy II', 'departmentId': 441, 'departmentName': 'FVT-K206', 'topic': '3. Syntéza kmitočtu ', 'topicId': 62645, 'masterId': '62645/2', 'timeslotsId': 1, 'timeslotsName': 'BR', 'lessonOrder': 8, 'lessonFormId': 2, 'lessonFormName': 'CV', 'lessonUnit': 45, 'lessonsCount': 2, 'groupsIds': ['A8CFD4D0-8CA4-11EB-BA48-520D00000000'], 'groupsNames': ['23-3LT-AV-C'], 'groupsEntryYearsIds': [], 'classroomsIds': [331], 'classroomsNames': ['Č1/120'], 'classroomsAreasIds': [2], 'teachersIds': [540], 'teachersNames': ['Němeček, Jiří'], 'teachersDepartmentsIds': [441], 'isLocked': True} ? subtopic
{'id': '2D2B1C90-F4FF-11EB-9B74-520D00000000', 'typeId': 'F00CB650-9768-11EB-87

> **Otázky**
>
> Jak můžeme naložit s hodnotami, které jsou vždy přítomné?
>
> Jak naložíme s hodnotami, které přítomné nejsou?

In [23]:
keys = {}
for item in events:
    for key in item.keys():
        value = item[key]
        if isinstance(value, dict):
            print(f'{key} is dict type / relation 1:1 expected?')
        if isinstance(value, list):
            print(f'{key} is list type / relation 1:N expected')
    break

startTime is dict type / relation 1:1 expected?
endTime is dict type / relation 1:1 expected?
date is dict type / relation 1:1 expected?
groupsIds is list type / relation 1:N expected
groupsNames is list type / relation 1:N expected
groupsEntryYearsIds is list type / relation 1:N expected
classroomsIds is list type / relation 1:N expected
classroomsNames is list type / relation 1:N expected
classroomsAreasIds is list type / relation 1:N expected
teachersIds is list type / relation 1:N expected
teachersNames is list type / relation 1:N expected
teachersDepartmentsIds is list type / relation 1:N expected


### Omezení dat na množinu vždy přítomných hodnot

In [25]:
def createDictLimiter(keys):
    def limiter(item):
        result = {}
        for key in keys:
            result[key] = item[key]
        return result
    return limiter


keyLimits = [
    'id', 'typeId', 'startTime', 'endTime', 'dateCode', 'date', 'categoryId', 
    'timeslotsId', 'timeslotsName', 'lessonUnit', 'groupsIds', 'groupsNames', 
    'groupsEntryYearsIds', 'classroomsIds', 'classroomsNames', 'classroomsAreasIds',
#    'teachersIds', 'teachersNames', 'teachersDepartmentsIds'
]

limiterFunc = createDictLimiter(keyLimits) 
for index, item in enumerate(map(limiterFunc, events)):
    print(item)
    if index > 5:
        break

{'id': '2D2B1C90-F4FF-11EB-9B74-520D00000000', 'typeId': 'F00CB650-9768-11EB-87D2-030800000000', 'startTime': {'hours': 14, 'minutes': 30}, 'endTime': {'hours': 16, 'minutes': 0}, 'dateCode': '2021-11-15', 'date': {'day': 15, 'month': 11, 'year': 2021}, 'categoryId': 1, 'timeslotsId': 1, 'timeslotsName': 'BR', 'lessonUnit': 45, 'groupsIds': ['A8CFD4D0-8CA4-11EB-BA48-520D00000000'], 'groupsNames': ['23-3LT-AV-C'], 'groupsEntryYearsIds': [], 'classroomsIds': [331], 'classroomsNames': ['Č1/120'], 'classroomsAreasIds': [2]}
{'id': 'C65A0170-F4FE-11EB-9B74-520D00000000', 'typeId': 'EFAE42A0-9768-11EB-87D2-030800000000', 'startTime': {'hours': 9, 'minutes': 50}, 'endTime': {'hours': 11, 'minutes': 20}, 'dateCode': '2021-10-19', 'date': {'day': 19, 'month': 10, 'year': 2021}, 'categoryId': 1, 'timeslotsId': 1, 'timeslotsName': 'BR', 'lessonUnit': 45, 'groupsIds': ['A8CFD4D0-8CA4-11EB-BA48-520D00000000'], 'groupsNames': ['23-3LT-AV-C'], 'groupsEntryYearsIds': [], 'classroomsIds': [332], 'class

## Heterogenní datové struktury

Heterogenní = různorodé, tzn. data mají různou strukturu. Blíže realitě. 

> **Otázka**
>
> Který typ dat (homogenní vs. heterogenní) byl v historii prvotní. Obhajujte svůj názor!

Data analyzována výše byla identifikována jako heterogenní. Pokud je budeme chtít uložit (persistence), máme různé možnosti, jedna z nich je homogenizace.

### Relace

Relace vyjadřují vztah mezi dvěma strukturami. Může se stát (a stává se), že datová struktura má vnořenou datovou strukturu (viz data výše).

V případě, kdy heterogenní data ukládáme v homogenních strukturách, je nutné provést homogenizaci / rozklad do relací:
- 1:1 (to ani není problém)
- 1:N (vnořená struktura je list)
- N:M (prvek vnořené struktury / listu / se vyskutuje i jinde)

## Operace nad datovými strukturami

### Filter

`filter` ([doc](https://docs.python.org/3/library/functions.html#filter)) je funkce která testuje, zda prvek splňuje kritérium specifikované pomocí funkce 

In [29]:
filterFunc = lambda item: '23-5KB' in item['groupsNames']
filteredEvents = filter(filterFunc, events)
print(filteredEvents)

> **Pozor**
>
> Výstup z funkce `filter` je generátor, takže iterace jej vyprázdní

In [28]:
for index, item in enumerate(filteredEvents):
    print(item)
    print('-'*30)
    if index > 5:
        break

{'id': '9FC2B630-070D-11EC-9D49-520D00000000', 'typeId': '56B77100-C398-11EB-9FF4-292400000000', 'startTime': {'hours': 18, 'minutes': 30}, 'endTime': {'hours': 20, 'minutes': 0}, 'dateCode': '2022-02-15', 'date': {'day': 15, 'month': 2, 'year': 2022}, 'categoryId': 1, 'subjectId': 625, 'subjectName': 'Aplikované vojenské technologie', 'departmentId': 380, 'departmentName': 'FVT-K201', 'topic': '6. Digitální geografická data', 'masterId': '3824/1', 'timeslotsId': 1, 'timeslotsName': 'BR', 'lessonOrder': 6, 'lessonFormId': 1, 'lessonFormName': 'P', 'lessonUnit': 45, 'lessonsCount': 2, 'groupsIds': ['67F34C60-7810-11EB-9A9C-520D00000000'], 'groupsNames': ['23-5KB'], 'groupsEntryYearsIds': [2019], 'classroomsIds': [193, 205], 'classroomsNames': ['Š5A/posl.', 'Š5B/posl.'], 'classroomsAreasIds': [5], 'teachersIds': [4340], 'teachersNames': ['Čapek, Jaromír'], 'teachersDepartmentsIds': [412], 'isLocked': True}
------------------------------
{'id': '31923F00-070D-11EC-9D49-520D00000000', 'typ

### Map

In [31]:
subjData = data['subjects']
subjData[0]

{'id': 33750, 'name': 'Aerobic', 'departmentId': 661}

In [34]:
def createSubjChecker(subjData):
    dataCopy = [*subjData]
    dataCopy.sort(key=lambda item: item['id'])
    ids = [item['id'] for item in dataCopy]
    names = [item['name'] for item in dataCopy]
    def checker(item):
        id = item['subjectId']
        index = ids.index(id)
        name = names[index]
        return {**item, 'subjNameOk': item['subjectName'] == name}
    return checker

checker = createSubjChecker(subjData)

checkedEvents = list(map(checker, events))
for item in checkedEvents:
    if not item['subjNameOk']:
        print(item)
        print('-'*30)

KeyError: 'subjectId'

In [35]:
filterFunc = lambda item: 'subjectId' in item
checkedEvents = list(map(checker, filter(filterFunc, events)))
for item in checkedEvents:
    if not item['subjNameOk']:
        print(item)
        print('-'*30)

### Reduce

`reduce` je funkce pracující nad listem (vektorem) dat a vytvářející skalární proměnnou.

In [36]:
from functools import reduce

arr = [0, 1, 2, 3]
reduceFunc = lambda acc, value: acc + value
result = reduce(reduceFunc, arr)
print(result)

6


Lze použít i na pole funkcí.

In [38]:
littlePartial = lambda func, firstParam: lambda secondParam: func(firstParam, secondParam)
def add(a, b):
    return a + b

add1 = littlePartial(add, 1)
result = add1(10)
print(result)

11


In [39]:
def funcArrayReductor(acc, func):
    return lambda param: func(acc(param))

funcArray = [
    littlePartial(filter, lambda item: 'subjectId' in item),
    littlePartial(map, checker),
    list
]
    
mainFunc = reduce(funcArrayReductor, funcArray)
checkedEvents = mainFunc(events)
print(checkedEvents[:1])

[{'id': '2D2B1C90-F4FF-11EB-9B74-520D00000000', 'typeId': 'F00CB650-9768-11EB-87D2-030800000000', 'startTime': {'hours': 14, 'minutes': 30}, 'endTime': {'hours': 16, 'minutes': 0}, 'dateCode': '2021-11-15', 'date': {'day': 15, 'month': 11, 'year': 2021}, 'categoryId': 1, 'subjectId': 32269, 'subjectName': 'Letecké elektronické systémy II', 'departmentId': 441, 'departmentName': 'FVT-K206', 'topic': '3. Syntéza kmitočtu ', 'topicId': 62645, 'masterId': '62645/2', 'timeslotsId': 1, 'timeslotsName': 'BR', 'lessonOrder': 8, 'lessonFormId': 2, 'lessonFormName': 'CV', 'lessonUnit': 45, 'lessonsCount': 2, 'groupsIds': ['A8CFD4D0-8CA4-11EB-BA48-520D00000000'], 'groupsNames': ['23-3LT-AV-C'], 'groupsEntryYearsIds': [], 'classroomsIds': [331], 'classroomsNames': ['Č1/120'], 'classroomsAreasIds': [2], 'teachersIds': [540], 'teachersNames': ['Němeček, Jiří'], 'teachersDepartmentsIds': [441], 'isLocked': True, 'subjNameOk': True}]


In [40]:
def funcArrayReduce(funcArray):
    def resultFunc(param):
        result = param
        for func in funcArray:
            result = func(result)
        return result
    return resultFunc

funcArray = [
    littlePartial(filter, lambda item: 'subjectId' in item),
    littlePartial(map, checker),
    list
]
    
mainFunc = funcArrayReduce(funcArray)
checkedEvents = mainFunc(events)
print(checkedEvents[:1])

[{'id': '2D2B1C90-F4FF-11EB-9B74-520D00000000', 'typeId': 'F00CB650-9768-11EB-87D2-030800000000', 'startTime': {'hours': 14, 'minutes': 30}, 'endTime': {'hours': 16, 'minutes': 0}, 'dateCode': '2021-11-15', 'date': {'day': 15, 'month': 11, 'year': 2021}, 'categoryId': 1, 'subjectId': 32269, 'subjectName': 'Letecké elektronické systémy II', 'departmentId': 441, 'departmentName': 'FVT-K206', 'topic': '3. Syntéza kmitočtu ', 'topicId': 62645, 'masterId': '62645/2', 'timeslotsId': 1, 'timeslotsName': 'BR', 'lessonOrder': 8, 'lessonFormId': 2, 'lessonFormName': 'CV', 'lessonUnit': 45, 'lessonsCount': 2, 'groupsIds': ['A8CFD4D0-8CA4-11EB-BA48-520D00000000'], 'groupsNames': ['23-3LT-AV-C'], 'groupsEntryYearsIds': [], 'classroomsIds': [331], 'classroomsNames': ['Č1/120'], 'classroomsAreasIds': [2], 'teachersIds': [540], 'teachersNames': ['Němeček, Jiří'], 'teachersDepartmentsIds': [441], 'isLocked': True, 'subjNameOk': True}]


## SQL / SQLAlchemy

In [ ]:
!pip install sqlalchemy

In [ ]:
from sqlalchemy import Column, String, Integer, ForeignKey
from sqlalchemy.orm import relationship, backref
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String(200), nullable=False)
    surname = Column(String(200), nullable=False)

## GraphQL (Graphene)